In [2]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 

In [3]:
master_df = pd.read_csv('C:/Users/Shoya/surf/data/master_df.csv', encoding='latin1')
display(master_df.head())
display(master_df.columns)

,Unnamed: 0,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Date(UTC),...,Close Price % Change,Close Price % Change (Abs),Is Price Anomaly (Bitmap Detector),Is Price Anomaly (Derivative Detector),Is Price % Change Anomaly (Bitmap Detector),Is Trend Anomaly (Bitmap Detector),Moving Average Price,Moving Average Price % Change,MA % Change (Abs.Value),Is Spike
0,0,1420840800,289.59,289.94,289.13,289.13,34.042422,9854.490388,289.476771,2015-01-09 22:00:00,...,NaN,NaN,0,0,0,0,NaN,NaN,NaN,0
1,1,1420844400,293.61,293.61,292.15,292.15,36.628012,10733.474260,293.040042,2015-01-09 23:00:00,...,0.010445,0.010445,0,0,0,0,NaN,NaN,NaN,1
2,2,1420848000,293.89,293.89,293.89,293.89,0.158015,46.439149,293.890000,2015-01-10 00:00:00,...,0.005956,0.005956,0,0,0,0,291.723333,NaN,NaN,0
3,3,1420851600,289.20,289.20,289.10,289.10,30.985828,8960.344674,289.175573,2015-01-10 01:00:00,...,-0.016299,0.016299,0,0,0,0,291.713333,-0.000034,0.000034,-1
4,4,1420855200,289.18,289.18,288.57,288.59,7.694195,2224.105007,289.062740,2015-01-10 02:00:00,...,-0.001764,0.001764,0,0,0,0,290.526667,-0.004068,0.004068,0


Index(['Unnamed: 0', 'Timestamp', 'Open', 'High', 'Low', 'Close',
       'Volume_(BTC)', 'Volume_(Currency)', 'Weighted_Price', 'Date(UTC)',
       'date', 'Bitcoin (Adj.Overlap)', 'isPartial', 'Date_UTC_trend',
       'Bitcoin (Unadj.)', 'Close Price % Change',
       'Close Price % Change (Abs)', 'Is Price Anomaly (Bitmap Detector)',
       'Is Price Anomaly (Derivative Detector)',
       'Is Price % Change Anomaly (Bitmap Detector)',
       'Is Trend Anomaly (Bitmap Detector)', 'Moving Average Price',
       'Moving Average Price % Change', 'MA % Change (Abs.Value)', 'Is Spike'],
      dtype='object')

In [4]:
df = master_df[['Timestamp', 'Close', 'Volume_(BTC)', 'Volume_(Currency)', 'Date(UTC)', 'Bitcoin (Adj.Overlap)', 
               'Close Price % Change', 'Close Price % Change (Abs)', 'Is Spike']]
display(df.head(2))

,Timestamp,Close,Volume_(BTC),Volume_(Currency),Date(UTC),Bitcoin (Adj.Overlap),Close Price % Change,Close Price % Change (Abs),Is Spike
0,1420840800,289.13,34.042422,9854.490388,2015-01-09 22:00:00,53.0,NaN,NaN,0
1,1420844400,292.15,36.628012,10733.474260,2015-01-09 23:00:00,52.0,0.010445,0.010445,1


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

# Preprocess


tscv = TimeSeriesSplit(n_splits=2)

X = df[['Timestamp', 'Close', 'Volume_(BTC)', 'Bitcoin (Adj.Overlap)', 'Close Price % Change']].values
Y = df['Is Spike'].values

train_size = int(len(X) * 0.75)
X_train, X_test = X[1:train_size], X[train_size:len(X)]
Y_train, Y_test = Y[1:train_size], Y[train_size:len(X)]
print('Observations: %d' % (len(X)))
print('Training Observations: %d' % (len(X_train)))
print('Testing Observations: %d' % (len(Y_test)))


display(X_train, Y_train)

Observations: 28083
Training Observations: 21061
Testing Observations: 7021


array([[  1.42084440e+09,   2.92150000e+02,   3.66280122e+01,
          5.20000000e+01,   1.04451280e-02],
       [  1.42084800e+09,   2.93890000e+02,   1.58015410e-01,
          5.20000000e+01,   5.95584500e-03],
       [  1.42085160e+09,   2.89100000e+02,   3.09858284e+01,
          5.60000000e+01,  -1.62986150e-02],
       ..., 
       [  1.49682600e+09,   2.82193000e+03,   3.88509000e-03,
          4.63711018e+02,  -4.33980300e-03],
       [  1.49682960e+09,   2.85101000e+03,   3.64904602e+00,
          4.42870073e+02,   1.03050040e-02],
       [  1.49683320e+09,   2.85896000e+03,   1.24272591e+00,
          4.11608656e+02,   2.78848500e-03]])

array([ 1,  0, -1, ...,  0,  1,  0], dtype=int64)

In [7]:
from tpot import TPOTClassifier 

tpot = TPOTClassifier(verbosity=2, max_time_mins=10,cv=tscv)
tpot.fit(X_train, Y_train)

Imputing missing values in feature set


10.011504383333333 minutes have elapsed. TPOT will close down.
TPOT closed prematurely. Will use the current best pipeline.



Best pipeline: RandomForestClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.35, min_samples_leaf=1, min_samples_split=7, n_estimators=100)


TPOTClassifier(config_dict={'sklearn.cluster.FeatureAgglomeration': {'linkage': ['ward', 'complete', 'average'], 'affinity': ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']}, 'sklearn.decomposition.FastICA': {'tol': array([ 0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,  0.4 ,
        0.45,  0.5 ,  0.55, ...lacian', 'polynomial', 'poly', 'linear', 'additive_chi2', 'sigmoid'], 'n_components': range(1, 11)}},
        crossover_rate=0.1,
        cv=TimeSeriesSplit(max_train_size=None, n_splits=2),
        disable_update_check=False, early_stop=None, generations=1000000,
        max_eval_time_mins=5, max_time_mins=10, memory=None,
        mutation_rate=0.9, n_jobs=1, offspring_size=100,
        periodic_checkpoint_folder=None, population_size=100,
        random_state=None, scoring=None, subsample=1.0, verbosity=2,
        warm_start=False)

In [9]:
tpot.score(X_test, Y_test)

0.99715140293405502

In [11]:
tpot.export('tpot_crypto_spike_predictor.py')

True

In [14]:
# The Result Code From TPOT
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

#tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
tpot_data = df[['Timestamp', 'Close', 'Volume_(BTC)', 'Bitcoin (Adj.Overlap)', 'Close Price % Change', 'Is Spike']]
features = tpot_data.drop('Is Spike', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['Is Spike'].values, random_state=42)

imputer = Imputer(strategy="median")
imputer.fit(training_features)
training_features = imputer.transform(training_features)
testing_features = imputer.transform(testing_features)

# Score on the training set was:0.9998575498575499
exported_pipeline = RandomForestClassifier(bootstrap=False, criterion="gini", max_features=0.35000000000000003, min_samples_leaf=1, min_samples_split=7, n_estimators=100)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


In [19]:
for i in range(100):
    print(testing_target[i], results[i])

0 0
0 0
0 0
0 0
1 1
0 0
0 0
0 0
-1 -1
0 0
0 0
0 0
1 1
1 1
0 0
0 0
-1 -1
1 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
-1 -1
0 0
0 0
1 1
0 0
0 0
0 0
0 0
0 0
-1 -1
0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
0 0
-1 -1
-1 -1
-1 -1
0 0
-1 -1
0 0
1 1
-1 -1
0 0
-1 -1
-1 -1
0 0
0 0
0 0
1 1
-1 -1
0 0
0 0
0 0
0 0
0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
0 0
-1 -1
0 0
0 0
0 0
0 0
